In [1]:
import os
import pandas as pd

##### 나중에 2021-07-02 데이터 없애고 합치는 부분 지우기

In [2]:
output_path = '../../data/concat_forecast_data'
os.makedirs(output_path, exist_ok=True)
concat_path = '../../data/concat_data'
weather_forecast_path = '../../data/weather_forecast_data/nextday_forecast'
folder = ['1400', '2000']

meta_data = pd.read_csv('../../data/solar_energy/meta_data.csv')
energy_files = os.listdir(concat_path)

for file_name in energy_files:
    print(f"\n===== 처리 중인 파일: {file_name} =====")
    
    meta_info = meta_data[meta_data['name'] == file_name.split('.')[0]]
    location_name = meta_info['location_name'].iloc[0]
    e_f = pd.read_parquet(os.path.join(concat_path, file_name))
    
    # e_f 데이터 기간 출력
    e_f_start = pd.to_datetime(e_f['date']).min()
    e_f_end = pd.to_datetime(e_f['date']).max()
    e_f_count = len(e_f)
    print(f"원본 에너지 데이터 기간: {e_f_start.date()} ~ {e_f_end.date()} (총 {e_f_count}개 레코드)")
    
    # 예보 데이터 읽기
    forecast_14 = pd.read_parquet(os.path.join(weather_forecast_path, folder[0], f'{location_name}.parquet'))
    forecast_20 = pd.read_parquet(os.path.join(weather_forecast_path, folder[1], f'{location_name}.parquet'))
        
    # 예보 데이터 기간 출력
    f14_start = pd.to_datetime(forecast_14['date']).min()
    f14_end = pd.to_datetime(forecast_14['date']).max()
    f14_count = len(forecast_14)
    print(f"14시 예보 데이터 기간: {f14_start.date()} ~ {f14_end.date()} (총 {f14_count}개 레코드)")
    
    f20_start = pd.to_datetime(forecast_20['date']).min()
    f20_end = pd.to_datetime(forecast_20['date']).max()
    f20_count = len(forecast_20)
    print(f"20시 예보 데이터 기간: {f20_start.date()} ~ {f20_end.date()} (총 {f20_count}개 레코드)")
    
    # 데이터 형식 변환
    e_f['date'] = pd.to_datetime(e_f['date']).dt.date
    forecast_14['date'] = pd.to_datetime(forecast_14['date']).dt.date
    forecast_20['date'] = pd.to_datetime(forecast_20['date']).dt.date
    e_f['time'] = pd.to_datetime(e_f['time'], format='%H:%M').dt.strftime('%H:%M')
    forecast_14['time'] = pd.to_datetime(forecast_14['time'], format='%H:%M').dt.strftime('%H:%M')
    forecast_20['time'] = pd.to_datetime(forecast_20['time'], format='%H:%M').dt.strftime('%H:%M')

    # 컬럼명 변경
    forecast_14 = forecast_14.rename(columns={
        '3시간기온': 'temp_14',
        '풍향': 'wd_14',
        '하늘상태': 'sc_14',
        '풍속': 'ws_14',
        '강수확률': 'pp_14'
    })

    forecast_20 = forecast_20.rename(columns={
        '3시간기온': 'temp_20',
        '풍향': 'wd_20',
        '하늘상태': 'sc_20',
        '풍속': 'ws_20',
        '강수확률': 'pp_20'
    })

    # 데이터 병합
    merged_data = pd.merge(
        e_f,
        forecast_14[['date', 'time', 'temp_14', 'wd_14', 'sc_14', 'ws_14', 'pp_14']],
        on=['date', 'time'],
        how='inner',
    )
    merged_data = pd.merge(
        merged_data,
        forecast_20[['date', 'time', 'temp_20', 'wd_20', 'sc_20', 'ws_20', 'pp_20']],
        on=['date', 'time'],
        how='inner',
    )
    
    merged_count = len(merged_data)
    
    # 병합 후 데이터가 있는 경우에만 처리
    if merged_count > 0:
        # 병합 후 데이터 기간 출력
        merged_start = pd.to_datetime(merged_data['date']).min()
        merged_end = pd.to_datetime(merged_data['date']).max()
        print(f"병합 후 데이터 기간: {merged_start} ~ {merged_end} (총 {merged_count}개 레코드)")
        
        try:
            print(f"데이터 감소율: {(e_f_count - merged_count) / e_f_count * 100:.2f}% 감소 ({e_f_count} -> {merged_count})")
        except ZeroDivisionError:
            print("데이터 감소율 계산 불가 (원본 데이터가 없습니다)")
        
        # 데이터가 있는 경우에만 저장
        merged_data.to_parquet(os.path.join(output_path, f'{file_name}'), index=False)
        print(f"{file_name} 처리 완료 - 저장 경로: {os.path.join(output_path, file_name)}")
    else:
        print(f"⚠️ 경고: 병합 후 데이터가 비어있습니다. {file_name}은(는) 저장되지 않았습니다.")
        print(f"세 데이터셋 간에 공통된 날짜/시간이 없는 것으로 보입니다.")


===== 처리 중인 파일: 부산운동장.parquet =====
원본 에너지 데이터 기간: 2018-01-01 ~ 2022-12-31 (총 43824개 레코드)
14시 예보 데이터 기간: 2015-01-02 ~ 2021-07-02 (총 56736개 레코드)
20시 예보 데이터 기간: 2015-01-02 ~ 2021-07-02 (총 56736개 레코드)
병합 후 데이터 기간: 2018-01-01 00:00:00 ~ 2021-07-02 00:00:00 (총 30696개 레코드)
데이터 감소율: 29.96% 감소 (43824 -> 30696)
부산운동장.parquet 처리 완료 - 저장 경로: ../../data/concat_forecast_data/부산운동장.parquet

===== 처리 중인 파일: 화촌주민참여형.parquet =====
원본 에너지 데이터 기간: 2021-08-01 ~ 2022-12-31 (총 12432개 레코드)
14시 예보 데이터 기간: 2015-01-02 ~ 2023-01-01 (총 69864개 레코드)
20시 예보 데이터 기간: 2015-01-02 ~ 2023-01-01 (총 69888개 레코드)
병합 후 데이터 기간: 2021-08-01 00:00:00 ~ 2022-12-31 00:00:00 (총 12432개 레코드)
데이터 감소율: 0.00% 감소 (12432 -> 12432)
화촌주민참여형.parquet 처리 완료 - 저장 경로: ../../data/concat_forecast_data/화촌주민참여형.parquet

===== 처리 중인 파일: 삼척소내_2.parquet =====
원본 에너지 데이터 기간: 2020-08-01 ~ 2022-12-31 (총 21192개 레코드)
14시 예보 데이터 기간: 2015-01-02 ~ 2023-01-01 (총 69864개 레코드)
20시 예보 데이터 기간: 2015-01-02 ~ 2023-01-01 (총 69888개 레코드)
병합 후 데이터 기간: 2020-08-01 00:00:00 ~ 

In [3]:
sample = pd.read_parquet('../../data/concat_forecast_data/부산본부_1.parquet')
print(len(sample[sample['temp_14'].isna()]))

sample

24


,date,호기,총량(kw),평균(kw),최대(kw),최소(kw),최대(시간별_kw),최소(시간별_kw),value,time,...,temp_14,wd_14,sc_14,ws_14,pp_14,temp_20,wd_20,sc_20,ws_20,pp_20
0,2015-01-02,1,625.932,26.000,133.596,0,133.596,9.144,0.000,00:00,...,-2.0,272.0,2.0,3.0,10.0,-2.0,279.0,2.0,4.0,10.0
1,2015-01-02,1,625.932,26.000,133.596,0,133.596,9.144,0.000,01:00,...,-1.0,272.0,2.0,3.0,10.0,-1.0,282.0,2.0,4.0,10.0
2,2015-01-02,1,625.932,26.000,133.596,0,133.596,9.144,0.000,02:00,...,1.0,273.0,2.0,4.0,10.0,0.0,286.0,2.0,4.0,10.0
3,2015-01-02,1,625.932,26.000,133.596,0,133.596,9.144,0.000,03:00,...,2.0,273.0,2.0,4.0,10.0,1.0,289.0,2.0,4.0,10.0
4,2015-01-02,1,625.932,26.000,133.596,0,133.596,9.144,0.000,04:00,...,3.0,282.0,2.0,4.0,10.0,2.0,289.0,2.0,4.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56755,2021-07-02,1,459.216,19.134,64.692,0,64.692,1.152,5.292,19:00,...,<NA>,<NA>,nan,<NA>,20.0,<NA>,<NA>,nan,<NA>,20.0
56756,2021-07-02,1,459.216,19.134,64.692,0,64.692,1.152,1.152,20:00,...,<NA>,<NA>,nan,<NA>,20.0,<NA>,<NA>,nan,<NA>,20.0
56757,2021-07-02,1,459.216,19.134,64.692,0,64.692,1.152,0.000,21:00,...,<NA>,<NA>,nan,<NA>,20.0,<NA>,<NA>,nan,<NA>,20.0
56758,2021-07-02,1,459.216,19.134,64.692,0,64.692,1.152,0.000,22:00,...,<NA>,<NA>,nan,<NA>,20.0,<NA>,<NA>,nan,<NA>,20.0
